In [4]:
import fitz  # PyMuPDF

def extract_lines_from_pdf_pymupdf(pdf_file):
    """
    Extrae las líneas completas de texto de un archivo PDF utilizando PyMuPDF.
    
    Args:
        pdf_path (str): Ruta del archivo PDF.
        
    Returns:
        list: Lista de líneas extraídas del PDF.
    """
    lines = []
    with fitz.open(pdf_file) as doc:
        for page in doc:
            text = page.get_text("text")  # Obtener texto como líneas completas
            page_lines = text.splitlines()
            lines.extend([line.strip() for line in page_lines if line.strip()])
    return lines

pdf_path = "./Tenis.pdf"  
pdf_lines = extract_lines_from_pdf_pymupdf(pdf_path)
print(pdf_lines)

['El tenis es un deporte que se disputa entre dos jugadores (individuales) o entre dos parejas', '(dobles).', 'El objetivo principal del juego es lanzar una pelota golpeándola con la raqueta de modo que', 'rebote en el otro lado pasando la red dentro de los límites permitidos del campo del rival,', 'procurando que este no pueda devolverla para conseguir un segundo rebote en el suelo y', 'darle un punto.2\u200b', 'Etimología', 'La palabra española «tenis» proviene del inglés «tennis», que a su vez tiene su origen en el', 'francés «tenez».3\u200bCuando el jugador de tenis (llamado «tenista») ponía la pelota en juego,', 'exclamaba «¡tenuz!» («¡ahí va!», en francés).4\u200b', 'Historia', 'Pista de césped en los EE. UU., 1887.', 'Final de dobles de tenis en los Juegos Olímpicos de 1896.', 'Artículo principal: Historia del tenis', 'Las primeras referencias del tenis tienen lugar en Francia, nombrado «le Paume» (‘juego de', 'palmas’), dado que al principio se golpeaba la pelota con la mano. M

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/home/bigdata/miniconda3/envs/rags/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_chroma import Chroma


vector_store = Chroma.from_texts(
    texts=pdf_lines,
    collection_name="pdf-prueba6",
    embedding=embeddings,
)

In [7]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="pdf-prueba6",
    embedding_function=embeddings,
)

In [8]:
retriever = vector_store.as_retriever()

In [9]:
question = "Que es el tenis?"
docs = vector_store.similarity_search(question)
len(docs)

4

In [10]:
docs[0]

Document(metadata={}, page_content='La red del tenis')

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM
ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

In [12]:
def search_chroma(input,history):
    try:
        # Invocar la cadena con la consulta como input
        response = chain.invoke(input)
        return response  
    except Exception as e:
        # Manejo de errores
        return f"Error al procesar la consulta: {e}"

In [15]:
# Definir la interfaz de chat
import gradio as gr
with gr.Blocks(theme=gr.themes.Glass()) as demo:
    gr.Markdown("### Chat con llama3.2")
    
    # Crear el ChatInterface para manejar la interacción
    chat_interface = gr.ChatInterface(fn=search_chroma,type="messages",
                                       examples=["Quien es Martin Scorsese", "Que relación tiene la violencia es sus películas?", "Háblame de Goodfellas"],
                                       )

# Lanzar la interfaz
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
